In [1]:
# 5 モンテカルロ法

In [10]:
import numpy as np

from collections import defaultdict

### 5.1.2 分布モデルとサンプルモデル

In [2]:
def sample(dices=2):
    x = 0
    for _ in range(dices):
        x += np.random.choice([1, 2, 3, 4, 5, 6])
    return x

In [3]:
print(sample())
print(sample())
print(sample())

5
7
3


### 5.1.3 モンテカルロ法の実装

In [4]:
trail = 1000

samples = []
for _ in range(trail):
    s = sample()
    samples.append(s)
    
V = sum(samples)/len(samples)
print(V)

7.073


In [5]:
trial = 1000

samples = []
for _ in range(trial):
    s = sample()
    samples.append(s)
    V = sum(samples)/len(samples)
    print(V)

7.0
7.5
6.333333333333333
5.5
6.4
6.833333333333333
6.571428571428571
6.375
6.222222222222222
6.6
6.636363636363637
6.416666666666667
6.769230769230769
6.785714285714286
6.866666666666666
6.6875
6.705882352941177
6.555555555555555
6.526315789473684
6.5
6.714285714285714
6.590909090909091
6.608695652173913
6.541666666666667
6.56
6.576923076923077
6.555555555555555
6.535714285714286
6.551724137931035
6.5
6.516129032258065
6.4375
6.484848484848484
6.4411764705882355
6.457142857142857
6.5
6.594594594594595
6.631578947368421
6.564102564102564
6.45
6.365853658536586
6.261904761904762
6.3023255813953485
6.318181818181818
6.355555555555555
6.413043478260869
6.361702127659575
6.416666666666667
6.428571428571429
6.5
6.549019607843137
6.5576923076923075
6.547169811320755
6.537037037037037
6.527272727272727
6.535714285714286
6.56140350877193
6.568965517241379
6.5423728813559325
6.6
6.622950819672131
6.645161290322581
6.619047619047619
6.671875
6.615384615384615
6.606060606060606
6.582089552238806


In [6]:
trial = 1000
V,n = 0,0

for _ in range(trial):
    s = sample()
    n+=1
    V += (s - V)/n          
    print(V)

8.0
6.0
6.0
5.75
6.8
6.333333333333333
6.714285714285714
6.625
6.555555555555555
6.7
6.909090909090909
6.916666666666667
6.769230769230769
7.142857142857142
7.066666666666666
7.0625
6.9411764705882355
6.833333333333334
6.7894736842105265
6.8
6.714285714285714
6.636363636363637
6.608695652173913
6.833333333333333
6.8
6.884615384615384
6.851851851851851
6.999999999999999
6.999999999999999
6.966666666666666
6.903225806451612
6.968749999999999
6.999999999999999
6.941176470588235
7.057142857142856
7.083333333333332
7.189189189189188
7.184210526315788
7.1025641025641
6.974999999999998
6.975609756097559
7.047619047619046
7.023255813953487
6.954545454545453
6.933333333333332
7.043478260869564
7.04255319148936
7.083333333333332
7.061224489795917
7.019999999999999
7.078431372549018
7.096153846153845
7.094339622641508
7.129629629629628
7.072727272727271
7.1428571428571415
7.122807017543859
7.137931034482758
7.101694915254237
7.1499999999999995
7.131147540983606
7.096774193548386
7.031746031746031

In [8]:
## 5.3 モンテカルロ法の実装

In [41]:

class GridWorld:
    def __init__(self):
        self.action_space = [0, 1, 2, 3]
        self.action_meaning = {
            0:"UP",
            1:"DOWN",
            2:"LEFT",
            3:"RIGHT"
        }
        
        self.reward_map = np.array([[0, 0, 0, 1.0],
                                    [0, None, 0, -1.0],
                                    [0, 0, 0, 0]])
        self.goal_state = (0, 3)
        self.wall_state = (1, 1)
        self.start_state = (2, 0)
        self.agent_state = self.start_state
        
    @property
    def height(self):
        return len(self.reward_map)
    
    @property
    def width(self):
        return len(self.reward_map[0])
    
    @property
    def shape(self):
        return self.reward_map.shape
    
    def actions(self):
        return self.action_space
    
    def states(self):
        for h in range(self.height):
            for w in range(self.width):
                yield (h, w)
                
    def next_state(self, state, action):
        action_move_map = [(-1, 0),(1,0),(0,-1),(0,1)]
        move = action_move_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        ny, nx = next_state
        
        if nx < 0 or nx >= self.width or ny < 0 or ny >= self.height:
            next_state = state
        elif next_state == self.wall_state:
            next_state = state
        
        return next_state
    
    def reward(self, next_state):
        return self.reward_map[next_state]
    
    def reset(self):
        self.agent_state = self.start_state
        return self.agent_state

    def step(self, action):
        state = self.agent_state
        next_state = self.next_state(state, action)
        reward = self.reward(next_state)
        done = (next_state == self.goal_state)

        self.agent_state = next_state
        return next_state, reward, done



### 5.3.1 stepメソッド

In [18]:
env = GridWorld()
action = 0
next_state, reward, done = env.step(action)

print('next_state:', next_state)
print('reward:', reward)
print('done:', done)

next_state: (1, 0)
reward: 0
done: False


In [9]:
### 5.3.2 エージェントクラスの実装

In [21]:
class RandomAgent:
    def __init__(self):
        self.gamma = 0.9
        self.action_size = 0.4
        
        random_actions = {0:0.25, 1:0.25, 2:0.25, 3:0.25}
        self.pi = defaultdict(lambda: random_actions)
        self.V = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []
    
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)
    
    def add(self, state, action, rewaed):
        data = (state, action, reward)
        self.memory.append(data)
    
    def reset(self):
        self.memory.clear()
        
    def eval(self):
        G = 0
        for data in reversed(self.memory):
            state, action, reward = data
            G = self.gamma*G + reward
            self.cnts[state] += 1
            self.V[state] += (G - self.V[state])/self.cnts[state]

In [13]:
### 5.3.3 モンテカルロ法を動かす

In [22]:
env = GridWorld()
agent = RandomAgent()

episodes = 1000
for episode in range(episodes):
    state = env.reset()
    agent.reset()
    
    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)
        
        agent.add(state, action ,reward)
        if done:
            agent.eval()
            break
        
        state = next_state

In [23]:
## 5.4 モンテカルロ法による方策制御

In [24]:
### 5.4.2 モンテカルロ法を使った方策制御の実装

In [28]:
class McAgent:
    def __init__(self):
        self.gamma = 0.9
        self.action_size =4
        
        random_actions = {0:0.25, 1:0.25, 2:0.25, 3:0.25}
        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []
    
    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)
    
    def add(self, state, action, rewaed):
        data = (state, action, reward)
        self.memory.append(data)
    
    def reset(self):
        self.memory.clear()
        
    def update(self):
        G = 0
        for data in reversed(self.memory):
            state, actin ,reward = data
            G = self.gamma *G + reward
            key = (state, action)
            self.cnts[key] += 1
            self.Q[key] += (G-self.Q[key])/self.cnts[key]
            
            self.pi[state] = greedy_probs(self.Q, state)

In [26]:
def greedy_probs(Q, state, action_size=4):
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = np.argmax(qs)
    
    action_probs = {action:0.0 for action in range(action_size)}
    action_probs[max] = 1
    return action_probs


In [29]:
### 5.4.3 ε-greedy法（1つ目の修正）

In [85]:
def greedy_probs(Q, state, epsilon=0, action_size=4):
    qs = [Q[(state, action)] for action in range(action_size)]
    max_action = np.argmax(qs)

    base_prob = epsilon / action_size
    action_probs = {action: base_prob for action in range(action_size)}  #{0: ε/4, 1: ε/4, 2: ε/4, 3: ε/4}
    action_probs[max_action] += (1 - epsilon)
    return action_probs


In [77]:
### 5.4.4 固定値α方式へ

In [78]:
### 5.4.5 モンテカルロ法を使った方策反復法の実装

In [86]:
class McAgent:
    def __init__(self):
        self.gamma = 0.9
        self.epsilon = 0.1
        self.alpha = 0.1
        self.action_size = 4

        random_actions = {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}
        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.memory = []

    def get_action(self, state):
        action_probs = self.pi[state]
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        return np.random.choice(actions, p=probs)

    def add(self, state, action, reward):
        data = (state, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def update(self):
        G = 0
        for data in reversed(self.memory):
            state, action, reward = data
            G = self.gamma * G + reward
            key = (state, action)
            self.Q[key] += (G - self.Q[key]) * self.alpha
            self.pi[state] = greedy_probs(self.Q, state, self.epsilon)

In [87]:
env = GridWorld()
agent = McAgent()

episodes = 10000
for episode in range(episodes):
    state = env.reset()
    agent.reset()

    while True:
        action = agent.get_action(state)
        next_state, reward, done = env.step(action)

        agent.add(state, action, reward)
        if done:
            agent.update()
            break

        state = next_state

In [88]:
agent.Q

defaultdict(<function __main__.McAgent.__init__.<locals>.<lambda>()>,
            {((0, 2), 3): 0.9999999999999996,
             ((0, 2), 0): 0.8929055533158076,
             ((0, 2), 1): 0.7777449881336036,
             ((0, 2), 2): 0.7920877989018311,
             ((1, 2), 0): 0.8948957003415434,
             ((1, 2), 1): 0.6275388380081685,
             ((1, 2), 2): 0.7000538568534715,
             ((1, 2), 3): -0.413283059970208,
             ((0, 1), 3): 0.8997889955971067,
             ((0, 1), 0): 0.7770353528512441,
             ((0, 1), 1): 0.7394681330096009,
             ((0, 1), 2): 0.7055533387269953,
             ((0, 0), 3): 0.7921413551823877,
             ((0, 0), 0): 0.6620782262283426,
             ((0, 0), 1): 0.5955076961214596,
             ((0, 0), 2): 0.6880084675961102,
             ((2, 2), 0): 0.7673166539157737,
             ((2, 2), 1): 0.3312966467763022,
             ((2, 2), 2): 0.5057886782857459,
             ((2, 2), 3): 0.6209088138093103,
          

In [55]:
## 5.5 方策オフ型と重点サンプリング 

In [56]:
### 5.5.2 重点サンプリング

In [58]:
x = np.array([1, 2, 3])
pi = np.array([0.1, 0.1, 0.8])

#期待値
e = np.sum(x*pi)
print('E_pi[x]', e)

#モンテカルロ法
n = 100
samples = []
for _ in range(n):
    s = np.random.choice(x, p=pi)
    samples.append(s)
    
mean = np.mean(samples)
var = np.var(samples)
print('MC: {:.2f} (var: {:.2f})'.format(mean, var))

E_pi[x] 2.7
MC: 2.62 (var: 0.54)


In [59]:
b = np.array([1/3, 1/3, 1/3])
n = 100
samples = []

for _ in range(n):
    idx = np.arange(len(b))
    i = np.random.choice(idx, p=b)
    s = x[i]
    rho = pi[i]/b[i]
    samples.append(rho*s)

mean = np.mean(samples)
var = np.var(samples)
print('IS: {:.2f}(var: {:.2f})'.format(mean, var))

IS: 2.68(var: 10.07)


In [60]:
### 5.5.3 分散を小さくするには

In [61]:
b = np.array([0.2, 0.2, 0.6])
n = 100
samples = []

for _ in range(n):
    idx = np.arange(len(b))
    i = np.random.choice(idx, p=b)
    s = x[i]
    rho = pi[i]/b[i]
    samples.append(rho*s)

mean = np.mean(samples)
var = np.var(samples)
print('IS: {:.2f}(var: {:.2f})'.format(mean, var))

IS: 2.79(var: 2.54)
